In [9]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

In [10]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [60]:
II = ID.kron(ID)
XX = X.kron(X)
YY = Y.kron(Y)
ZZ = Z.kron(Z)
IZ = ID.kron(Z)
ZI = Z.kron(ID)
sig_yy = [YY.get()[i,i] for i in range(len(YY.get()))]
sig_iz = [IZ.get()[i,i] for i in range(len(IZ.get()))]
sig_zi = [ZI.get()[i,i] for i in range(len(ZI.get()))]
sig_iz
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [61]:
H = II*2 + YY + IZ*3 - ZI + ZZ*2

min(scipy.linalg.eig(H.get())[0])

(-4.123105625617661+0j)

In [62]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])

In [59]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761]]), array([-4.12310563, -4.12310563, -4.12310563, -4.12310563, -4.12310563,
       -4.12310563, -4.12310563]))
           fun: -4.123105625617663
       message: 'Optimization terminated successfully.'
          nfev: 899
           nit: 518
        status: 0
       success: True
             x: array([-1.08588037, -1.

In [73]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_iz) # [zz, iz] = 0
    zi = expectation_value(result, sig_zi) # [zz, zi] = 0
    return zz, iz, zi

def expectation_energy(params):
    #xx =  sigma_xx(params)
    yy =  sigma_yy(params)
    zz, iz, zi =  sigma_zz(params)

    result = 2 + yy + 3*iz - zi + 2*zz
    return result

In [74]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882]]), array([-4.12310563, -4.12310563, -4.12310563, -4.12310563, -4.12310563,
       -4.12310563, -4.12310563]))
           fun: -4.123105625617663
       message: 'Optimization terminated successfully.'
          nfev: 991
           nit: 574
        status: 0
       success: True
             x: array([ 1.71336166, -3.

In [75]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [78]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [79]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_14932/1088512639.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


1.7006599878076414
1.2545549614764386
1.1497605201879366
0.22736544200075426
0.0978543078026497
0.017160441604664012
0.18828757124844753
0.2610390239473779
0.5487657730870437
0.8427397298047863
1.1747962716587876
1.175037952625668
1.2701384756107241
0.9374663154575005
1.0295511077500044
0.6964526889194024
0.6839972299070306
0.46189779463680064
0.39424493769346963
0.2737567403818101
0.21726307543124623
0.15446090748048297
0.118717492097142
0.08565273277681701
0.06464332044561588
0.046960241224349886
0.03499562474990564
0.025460538146252987
0.018780133747339722
0.013631896309170555
0.009958528103665376
0.007189234555726842
0.005197279928212284
0.003719269828432792
0.0026546739228606175
0.0018751334865257374
0.0013156704131106478
0.0009112873837680557


In [80]:
expectation_energy(params)

(-4.123095913489123+0j)